In [1]:
import torch
import torch.nn as nn
from transformers import AutoModel
import pandas as pd

import gc
import torch as th
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
import transformers
import transformers as tfms
import torch.optim as optim
import matplotlib.pyplot as plt
from typing import Tuple
from datasets import load_dataset
import os 


os.environ['TOKENIZERS_PARALLELISM'] = 'false'

torch.manual_seed(0)
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
try:
  FLOAT = torch.bfloat32 if torch.cuda.is_bf16_supported() else \
    torch.float32
except Exception as e:
  print(e)
  FLOAT = torch.float32
print(DEVICE, FLOAT)

cuda torch.float32


In [2]:
# config = tfms.DistilBertConfig.from_pretrained('hf-internal-testing/tiny-random-distilbert')
# model = tfms.AutoModelForMaskedLM.from_config(config)
model = tfms.AutoModelForMaskedLM.from_pretrained('hf-internal-testing/tiny-random-distilbert')

Some weights of the model checkpoint at hf-internal-testing/tiny-random-distilbert were not used when initializing DistilBertForMaskedLM: ['qa_outputs.bias', 'pre_classifier.bias', 'classifier.bias', 'pre_classifier.weight', 'qa_outputs.weight', 'classifier.weight']
- This IS expected if you are initializing DistilBertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
eli5 = load_dataset("eli5", split="train_asks[:5000]")

Reusing dataset eli5 (/home/tornikeo/.cache/huggingface/datasets/eli5/LFQA_reddit/1.0.0/17574e5502a10f41bbd17beba83e22475b499fa62caa1384a3d093fc856fe6fa)


In [4]:
eli5 = eli5.train_test_split(test_size=0.2)

In [5]:
eli5["train"][0]

{'q_id': '26rez4',
 'title': 'Do white/red dwarfs have a Habitable Zone?',
 'selftext': 'If a planet for all intents and purposes identical to Earth were to orbit around a white or red dwarf, would the "dead" star be able to supply enough energy to sustain life?\n\n\nIf they can, where would the zone be, and how long could the star provide for it\'s hypothetical life-bearing planet?',
 'document': '',
 'subreddit': 'askscience',
 'answers': {'a_id': ['chu6qmj'],
  'text': ['First off, a white dwarf is a "dead" star, while a red dwarf is just a small normal star. \n\nWhat we call a habitable zone is just the zone in which water on the surface would be liquid on a planet with sufficient atmospheric pressure. Because these dwarfs give off less energy than the sun, the zone is closer to the star.\n\nRed dwarfs would be able to provide life longer than the sun could, because red dwarfs live longer.\n\nWhite dwarfs most likely won\'t support life because they cool down from when they form, w

In [6]:
tokenizer = tfms.AutoTokenizer.from_pretrained('hf-internal-testing/tiny-random-distilbert')

In [7]:
eli5 = eli5.flatten()
eli5['train'][0]

{'q_id': '26rez4',
 'title': 'Do white/red dwarfs have a Habitable Zone?',
 'selftext': 'If a planet for all intents and purposes identical to Earth were to orbit around a white or red dwarf, would the "dead" star be able to supply enough energy to sustain life?\n\n\nIf they can, where would the zone be, and how long could the star provide for it\'s hypothetical life-bearing planet?',
 'document': '',
 'subreddit': 'askscience',
 'answers.a_id': ['chu6qmj'],
 'answers.text': ['First off, a white dwarf is a "dead" star, while a red dwarf is just a small normal star. \n\nWhat we call a habitable zone is just the zone in which water on the surface would be liquid on a planet with sufficient atmospheric pressure. Because these dwarfs give off less energy than the sun, the zone is closer to the star.\n\nRed dwarfs would be able to provide life longer than the sun could, because red dwarfs live longer.\n\nWhite dwarfs most likely won\'t support life because they cool down from when they form

In [8]:
eli5["train"].column_names

['q_id',
 'title',
 'selftext',
 'document',
 'subreddit',
 'answers.a_id',
 'answers.text',
 'answers.score',
 'title_urls.url',
 'selftext_urls.url',
 'answers_urls.url']

In [9]:
eli5.column_names

{'train': ['q_id',
  'title',
  'selftext',
  'document',
  'subreddit',
  'answers.a_id',
  'answers.text',
  'answers.score',
  'title_urls.url',
  'selftext_urls.url',
  'answers_urls.url'],
 'test': ['q_id',
  'title',
  'selftext',
  'document',
  'subreddit',
  'answers.a_id',
  'answers.text',
  'answers.score',
  'title_urls.url',
  'selftext_urls.url',
  'answers_urls.url']}

In [10]:
def preprocess_function(examples):
  x = tokenizer([' '.join(ex) for ex
                   in examples['answers.text']],
                   truncation=True,
                   padding=True,
#                    return_tensors='np'
                  )
  print({k:(len(v),len(v[0]), 'to', len(v[-1])) for k,v in x.items()})
  return x

tokenized_eli5 = eli5.map(
    preprocess_function,
    batched=True,
    remove_columns=eli5["train"].column_names,
#     num_proc=4,
)

Parameter 'function'=<function preprocess_function at 0x7f7d331b0820> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/4 [00:00<?, ?ba/s]

{'input_ids': (1000, 512, 'to', 512), 'attention_mask': (1000, 512, 'to', 512)}
{'input_ids': (1000, 512, 'to', 512), 'attention_mask': (1000, 512, 'to', 512)}
{'input_ids': (1000, 512, 'to', 512), 'attention_mask': (1000, 512, 'to', 512)}
{'input_ids': (1000, 512, 'to', 512), 'attention_mask': (1000, 512, 'to', 512)}


  0%|          | 0/1 [00:00<?, ?ba/s]

{'input_ids': (1000, 512, 'to', 512), 'attention_mask': (1000, 512, 'to', 512)}


In [11]:
tokenized_eli5

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 1000
    })
})

In [12]:
block_size=128
def group_texts(examples):
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
  

lm_dataset = tokenized_eli5.map(group_texts, 
                                batched=True,) 

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [13]:
# block_size = 128

# z = None
# def group_texts(examples):
# #   global z
#   #     print({k:(len(v),len(v[0])) for k,v in examples.items()})
#   #     print({k:v[0][-1] for k,v in examples.items()})
# #   try:
# #     concatenated_examples = {k: sum(examples[k], []) 
# #                              for k in examples.keys()}
#     print(list(examples.keys())[:4])
#     concatenated_examples = {k: sum(examples[k], []) 
#                              for k in examples.keys()}
#     #     print({k:(len(v)) for k,v in concatenated_examples.items()})
#     total_length = len(concatenated_examples[list(examples.keys())[0]])
#     print({k:(len(v)) for k,v in concatenated_examples.items()})

#     result = {
#         k: [t[i : i + block_size] 
#             for i in range(0, total_length, block_size)]
#         for k, t in concatenated_examples.items()
#     }
#     result["labels"] = result["input_ids"].copy()
#     return result

# #   except TypeError as e:
# #     z = examples
# # #     raise e
# #     print(e)
# #     return None
# lm_dataset = tokenized_eli5.map(group_texts, 
#                                 batched=True,) 
# #                                 num_proc=4)

In [14]:
# x = tfms.AutoTokenizer.from_pretrained('distilbert-base-cased')

In [15]:
tokenizer.special_tokens_map

{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

In [16]:
# tokenizer.pad_token = '[PAD]'
data_collator = tfms.DataCollatorForLanguageModeling(
  tokenizer=tokenizer,
  mlm_probability=.15,
)

In [17]:
np.array(lm_dataset['test'][:5]['input_ids']).shape

(5, 128)

In [18]:
tfms.TrainingArguments??

In [19]:
tfms.Trainer??

In [42]:
model = tfms.AutoModelForMaskedLM.from_pretrained('hf-internal-testing/tiny-random-distilbert')

loading configuration file https://huggingface.co/hf-internal-testing/tiny-random-distilbert/resolve/main/config.json from cache at /home/tornikeo/.cache/huggingface/transformers/825fc05fc9603996d64ce61190f3b2eadb28e7a2f4db70d2cb44c8b5457deab6.5cd95c0833b01050f80cc06f242975c6b324790c205343941ec863daed8f33c8
Model config DistilBertConfig {
  "_name_or_path": "hf-internal-testing/tiny-random-distilbert",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 32,
  "dropout": 0.1,
  "hidden_act": "gelu",
  "hidden_dim": 37,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 4,
  "n_layers": 5,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "transformers_version": "4.20.1",
  "vocab_size": 1124
}

loading weights file https://huggingface.co/hf-internal-testing/tiny-random-distilbert/resolve/main/pytorch_

In [45]:
!rm -rf results/*
training_args = tfms.TrainingArguments(
  output_dir='./results',
#   evaluation_strategy="epoch",
  evaluation_strategy="steps",
  eval_steps=100,
  learning_rate=2e-9,
  num_train_epochs=3,
  weight_decay=0.01,
)

trainer = tfms.Trainer(
  model=model,
  args=training_args,
  train_dataset=lm_dataset['train'],
  eval_dataset=lm_dataset['test'],
  data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [49]:
x = lm_dataset['train'][:5]
x = {k:torch.tensor(v, device='cuda') for k,v in x.items()}
y = model(**x)
y

MaskedLMOutput(loss=tensor(nan, device='cuda:0', grad_fn=<NllLossBackward0>), logits=tensor([[[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         ...,
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan],
         [nan, nan, nan,  ..., nan, nan, nan]],

        [[nan, nan, n

In [48]:
trainer.train()

***** Running training *****
  Num examples = 16000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 6000


Step,Training Loss,Validation Loss
100,No log,nan


***** Running Evaluation *****
  Num examples = 4000
  Batch size = 8


KeyboardInterrupt: 